In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

In [2]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

df_analyse.sort_values('date',ascending=True).tail()

,date,India,US,Spain,Germany,United Kingdom,"Korea, South",Japan
231,2020-09-09,4465863,6360212,543379,256433,357613,21743,73264
232,2020-09-10,4562414,6396100,554143,258149,360544,21919,73916
233,2020-09-11,4659984,6443652,566326,259735,364088,22055,74558
234,2020-09-12,4754356,6485123,566326,260817,367592,22176,75206
235,2020-09-13,4846427,6519573,566326,261737,370930,22285,75646


## HELPER FUNCTIONS


In [3]:
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()
      
    

In [4]:
threshold=100
compare_list=[]
for pos,country in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))


In [5]:
df_analyse.columns[1:]

Index(['India', 'US', 'Spain', 'Germany', 'United Kingdom', 'Korea, South',
       'Japan'],
      dtype='object')

In [6]:
pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T


In [7]:
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])
pd_sync_timelines.tail()

,India,US,Spain,Germany,United Kingdom,"Korea, South",Japan,date
202,NaN,NaN,NaN,NaN,NaN,21743.0,73916.0,202
203,NaN,NaN,NaN,NaN,NaN,21919.0,74558.0,203
204,NaN,NaN,NaN,NaN,NaN,22055.0,75206.0,204
205,NaN,NaN,NaN,NaN,NaN,22176.0,75646.0,205
206,NaN,NaN,NaN,NaN,NaN,22285.0,NaN,206


In [8]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

In [9]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

In [11]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [12]:
max_days=pd_sync_timelines.shape[0]

norm_slopes={
    #'doubling every day':doubling_rate(100,np.arange(max_days),1),
    #'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
    'doubling every 15 days':doubling_rate(100,np.arange(max_days),15),
    'doubling every 20 days':doubling_rate(100,np.arange(max_days),20),

}

In [13]:
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)

In [14]:
pd_sync_timelines_w_slope.head()

,doubling every 4 days,doubling every 10 days,doubling every 15 days,doubling every 20 days,India,US,Spain,Germany,United Kingdom,"Korea, South",Japan,date
0,100.000000,100.000000,100.000000,100.000000,102.0,103.0,120.0,130.0,134.0,104.0,112.0,0
1,118.920712,107.177346,104.729412,103.526492,113.0,172.0,165.0,159.0,189.0,204.0,137.0,1
2,141.421356,114.869835,109.682498,107.177346,119.0,215.0,222.0,196.0,246.0,433.0,149.0,2
3,168.179283,123.114441,114.869835,110.956947,142.0,337.0,259.0,262.0,295.0,602.0,160.0,3
4,200.000000,131.950791,120.302504,114.869835,156.0,450.0,400.0,482.0,374.0,833.0,173.0,4


In [15]:
#pd_sync_timelines_w_slope['doubling every day'].astype(np.int64)


In [16]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:9],
           y_scale='log',
           slider=True)

In [17]:
pd_sync_timelines_w_slope.to_csv('../data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

## Linear Regression

In [18]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)


In [19]:
X=np.arange(df_analyse['India'][df_analyse['India'] != 0].shape[0]).reshape(-1,1)
y=np.log(np.array(df_analyse['India'][df_analyse['India'] != 0]))

In [20]:
reg.fit(X, y)

LinearRegression(fit_intercept=False)

In [21]:
df_analyse['India'][120]

118226

In [22]:
reg.predict(np.array([120]).reshape(-1,1))


array([10.00521177])

In [23]:
X_hat=np.arange(df_analyse.shape[0]).reshape(-1, 1)
Y_hat=reg.predict(X_hat)

In [24]:
LR_inspect=df_analyse[['date','India']].copy()
LR_inspect['prediction']=np.exp(Y_hat)

In [25]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

## Doubling Rate
### Piecewise Linear Regression

In [26]:
from sklearn import linear_model
from scipy import signal

reg = linear_model.LinearRegression(fit_intercept=True)

In [27]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  
country_list=df_analyse.columns[1:]

In [28]:
X=np.arange(df_analyse['India'][df_analyse['India'] != 0].shape[0]).reshape(-1,1)
y=np.array(df_analyse['India'][df_analyse['India'] != 0])

In [29]:
reg.fit(X,y)

LinearRegression()

In [30]:
print(reg.coef_)
print(reg.intercept_)

[15593.03060464]
-954442.7674864015


In [31]:
reg.coef_/reg.intercept_

array([-0.01633731])

In [32]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope



def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [33]:
df_analyse_DR = df_analyse.copy()

In [34]:
# run on all filtered data
days_back = 3 # this gives a smoothing effect

In [35]:
df_analyse_DR['India_DR'] = df_analyse_DR['India'].rolling(window=days_back, min_periods=days_back).apply(get_doubling_time_via_regression)

In [36]:
quick_plot(df_analyse_DR.date,
           df_analyse_DR.iloc[40:,[8]],
           y_scale='linear',
           slider=True)

In [37]:
df_analyse_DR['India_DR_formulated'] = df_analyse_DR['India'].rolling(window=days_back, min_periods=days_back).apply(doubling_time) 

In [38]:
quick_plot(df_analyse_DR.date,
           df_analyse_DR.iloc[40:,[8,9]],
           y_scale='linear',
           slider=True)

In [39]:
df_analyse_DR_all = df_analyse.copy()

In [40]:
country_list= df_analyse_DR_all.columns[1:]
for each in country_list:
    df_analyse_DR_all[each+'_DR'] = df_analyse_DR_all[each].rolling(window=days_back, min_periods=days_back).apply(doubling_time).round(4)


In [41]:
quick_plot(df_analyse_DR_all.date,
           df_analyse_DR_all.iloc[50:,len(df_analyse.columns):],
           y_scale='log',
           slider=True)

In [42]:
df_analyse_filter = df_analyse.copy()

In [43]:
window_size = 5
for each in country_list:

    df_analyse_filter[each+'_filter'] = signal.savgol_filter(df_analyse_filter[each], 
                                                        window_length=window_size, #Window size for filtering
                                                        polyorder=1 #Order of polynomial approximation
                                                        )

In [44]:
filter_list = df_analyse_filter.columns[len(df_analyse.columns) : ]

In [45]:
  df_analyse_filter.head()

,date,India,US,Spain,Germany,United Kingdom,"Korea, South",Japan,India_filter,US_filter,Spain_filter,Germany_filter,United Kingdom_filter,"Korea, South_filter",Japan_filter
0,2020-01-22,0,1,0,0,0,1,2,0.0,0.4,0.0,0.0,0.0,0.8,1.6
1,2020-01-23,0,1,0,0,0,1,2,0.0,1.3,0.0,0.0,0.0,1.3,2.0
2,2020-01-24,0,2,0,0,0,2,2,0.0,2.2,0.0,0.0,0.0,1.8,2.4
3,2020-01-25,0,2,0,0,0,2,2,0.0,3.0,0.0,0.2,0.0,2.4,2.8
4,2020-01-26,0,5,0,0,0,3,4,0.0,3.8,0.0,1.0,0.0,3.0,3.8


In [46]:
quick_plot(df_analyse_filter.date,
           df_analyse_filter.iloc[50:,len(df_analyse.columns):],
           y_scale='log',
           slider=True)

In [47]:
for filter in filter_list:
    
    df_analyse_DR_all[filter+'_DR'] = df_analyse_filter[filter].rolling(window=days_back, min_periods=days_back).apply(doubling_time).round(4)

In [48]:
list(df_analyse_DR_all.columns)

['date',
 'India',
 'US',
 'Spain',
 'Germany',
 'United Kingdom',
 'Korea, South',
 'Japan',
 'India_DR',
 'US_DR',
 'Spain_DR',
 'Germany_DR',
 'United Kingdom_DR',
 'Korea, South_DR',
 'Japan_DR',
 'India_filter_DR',
 'US_filter_DR',
 'Spain_filter_DR',
 'Germany_filter_DR',
 'United Kingdom_filter_DR',
 'Korea, South_filter_DR',
 'Japan_filter_DR']

In [49]:
quick_plot(df_analyse_DR_all.date,
           df_analyse_DR_all.iloc[50:,[list(df_analyse_DR_all.columns).index("India_DR"), list(df_analyse_DR_all.columns).index("India_filter_DR")]],
           y_scale='linear',
           slider=True)

In [50]:
quick_plot(df_analyse_DR_all.date,
           df_analyse_DR_all.iloc[50:,len(df_analyse.columns)*2 - 1:],
           y_scale='linear',
           slider=True)